Clustering simulated sequences using TMRCA

Name: CQS21 
FYP 2024

Step 1. Generate burn in simulations establishing nucleotide diversity across a range of parameters (redo theo's work but w updated vgsc parameters)

#calculate position of vgsc gene
Its on q arm of chromosome 3 (AgamP4 assembly chromosome 3 is roughly 41.2 million bp in length)
The resistance mutations are found in the voltage gated sodium channel gene  (vgsc).
VGSC gene size is 6231bp
The  10kb long  haplotypes  began 5kb  upstream and
downstream of the resistance mutations: 2,417,652bp– 2,427,652bp. (10kb)


#position of rdl
#Rdl begins at 25,363,652bp and ends at 25,434,556 on chromosome 2L. A
missense mutation from cysteine to guanine causes an amino acid change from alanine to 
glycine at  codon  296 (A296G),  which  is  responsible  for  dieldrin  resistance (Du  et  al., 
2005). The position of the missense mutation was identifie by aligning a 213bp partial 
coding  sequence  containing  the  mutation (Accession  number:  AY787484)   (Du  et  al., 
2005), to the Agam P3 reference genome; the only mismatch between the sequences was 
at the position of the mutation. The presence of other non-synonymous mutations in rdl
occurring at a frequency greater than 5% in at least one of the sampled populations was 
recorded by filtering the Ag1000G data.

In [ ]:
## use HPC
1. SHELL SCRIPT for HPC BELOW #run test for just 1 burn in
#---------------------------------------------------------------------------------------------------------
!/bin/bash
PBS -N run_pairwise    ###REMOVE ME -  Change this to set the name of your script (can i set a script in SLIM?)
PBS -j oe
PBS -k oe             

PBS -m ae

PBS -l walltime=48:00:00
PBS -l select=1:ncpus=12:mem=5gb          #job parameters update after trial
                                  
module load anaconda3                     #loading modules: what modules do I need? is there a module for slim?
source activate f2                       # what does this f2 mean?


## to direct output to cwd, use $PBS_O_WORKDIR:
## specify LOGFILE found in ~/ during execution then moved to cwd on job completion
##
cd $PBS_O_WORKDIR
JOBNUM=`echo $PBS_JOBID | sed 's/\..*//'`
LOGFILE=${PBS_JOBNAME}.o${JOBNUM}

#########################################
##                                     ##
## Output some useful job information. ##
##                                     ##
#########################################

echo ------------------------------------------------------
echo -n 'Job is running on node '; cat $PBS_NODEFILE
echo ------------------------------------------------------
echo PBS: qsub is running on $PBS_O_HOST
echo PBS: originating queue is $PBS_O_QUEUE
echo PBS: executing queue is $PBS_QUEUE
echo PBS: working directory is $PBS_O_WORKDIR
echo PBS: execution mode is $PBS_ENVIRONMENT
echo PBS: job identifier is $PBS_JOBID
echo PBS: job name is $PBS_JOBNAME
echo PBS: job number is $JOBNUM
echo PBS: logfile is $LOGFILE
echo PBS: node file is $PBS_NODEFILE
echo PBS: current home directory is $PBS_O_HOME
#echo PBS: PATH = $PBS_O_PATH
echo ------------------------------------------------------


# move LOGFILE to cwd
##
mv $HOME/$LOGFILE $PBS_O_WORKDIR



#--------------------------------------------------------------------------------------------------------
2. submit shell script using resource manager
qsub slim_simulations.py

In [ ]:
#python script per burn in
#input = np.loa


#using subprocess module to run slim??
import subprocess


#python? script to iterate through all the burn ins
input = np.loadtxt("parameter_combinations.txt", dtype='f', delimiter=',')
counter = 0
array_index = int(os.environ['PBS_ARRAY_INDEX'])
for i in range(0,input.shape[0]):
    for j in range(0, input.shape[1]):
        counter = counter+1
        if counter == array_index:
            print('processing element', input[i,j])
            # your code goes in here
            
            # how to get it to run in SLIM???
             # Open the SLiM file in read mode
            with open("burnin_no.1.slim", "r") as file:
                # Read the contents of the file line by line
                for line in file:
                    #Process each line as needed
                    print(line.strip())  # Example: Print each line (stripping newline characters)

                    # Define the command to run SLiM with your SLiM script
            slim_command = ["slim", "-s", "burnin_no.1.slim"]

            # Execute the SLiM command
            try:
                subprocess.run(slim_command, check=True)
                print("SLiM simulation completed successfully!")
            except subprocess.CalledProcessError as e:
                print("Error: SLiM simulation failed:", e)

Step 2. Conduct 27 variations of partial soft sweep simulations and output vcf files using the HPC facility. Complete x number of repeats.

Size of population, N: 100, 1000, 10000

Mutation rate, μ : (0.1/4N), (1/4N), (5/4N)

Recombination rate, r : (0.1/4N), (1/4N), (5/4N)

In [6]:
import numpy as np

# Define parameter values
N_values = [100, 1000, 10000]
mutation_rates = [0.1 / (4 * N) for N in N_values]
recombination_rates = [0.1 / (4 * N) for N in N_values]

# Generate combinations of parameters
parameter_combinations = [(N, mu, r) for N in N_values for mu in mutation_rates for r in recombination_rates]

# Ensure we have exactly 27 combinations
assert len(parameter_combinations) == 27, "Number of combinations must be 27"

# Assign file numbers to combinations
burn_in_files = list(range(1, 28))  # List of file numbers from 1 to 27
combinations_matrix = np.zeros(27, dtype=[('N', int), ('mu', float), ('r', float), ('file', int)])
for index, (N, mu, r) in enumerate(parameter_combinations):
    combinations_matrix[index] = (N, mu, r, burn_in_files[index])

# Save the matrix to a text file
np.savetxt("parameter_combinations.txt", combinations_matrix, fmt="%d,%.6f,%.6f,%d", delimiter=",",
           header="N, mu, r, file", comments="")



In [ ]:
#DO NOT RUN 
1. SHELL SCRIPT for HPC BELOW

!/bin/bash
PBS -l walltime=24:00:00
PBS -l select=1:ncpus=30:mem=2gb          #job parameters update after trial
PBS -J 1-10                                  #array job for 27 combinations?
module load    ????                      #loading modules: what modules do I need? is there a module for slim?
cd $PBS_O_WORKDIR
slim_simulations.py                                 # my script in python


2. submit shell script using resource manager
qsub slim_simulations.py



##PBS -J n-m

#in your jobscript, where n and m are integers, e.g 1-4 if you wanted four runs. 
#You can then supply these to your slim command using the variable $PBS_ARRAY_INDEX with this taking on the value of J. 
#You'll then need something in your slim code to interpret and convert these values.



# choosing a wall time 
# Sensible values are 8, 24 hours, 48 hours or 72 hours. There is little benefit in selecting intermediate values.
#If possible, use checkpointing, i.e. save intermediate results that can be used to restart your program.



In [ ]:
#shell script for array run
#!/bin/bash

input="parameter_combinations.txt"

# Get the row number based on PBS_ARRAY_INDEX environment variable
array_index=$PBS_ARRAY_INDEX

# Read the specified row from the parameter combinations file
params=$(sed -n "${array_index}p" "$input")

# Split the row into individual parameters
IFS=',' read -r -a params_array <<< "$params"

# Extract parameters and process them
counter=0
for param in "${params_array[@]}"
do
    ((counter++))
    echo "Processing element ${counter}: ${param}"
    # Run your SLIM script with the parameters here
    slim -d seed=$PBS_ARRAY_INDEX  ~/simulations/sweep simulation.txt
done



# python script for matrix iteration? with array run
#The code uses $PBS_ARRAY_INDEX to execute the correct iteration of the loop that processes just one element of the matrix.
#All array runs combined process all elements of the matrix in parallel.
#slide 97 of the Intro to HPC wiki pdf
input = np.loadtxt("parameter_combinations.txt", dtype='f', delimiter=',')
counter = 0
array_index = int(os.environ['PBS_ARRAY_INDEX'])
for i in range(0,input.shape[0]):
    for j in range(0, input.shape[1]):
        counter = counter+1
        if counter == array_index:
            print('processing element', input[i,j])
            
            # Open the SLiM file in read mode
            with open("sweep simulation.slim", "r") as file:
                # Read the contents of the file line by line
                for line in file:
                    #Process each line as needed
                    print(line.strip())  # Example: Print each line (stripping newline characters)


            #SLIM Script below

